In [1]:
!pip install transformers
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
MODEL_NAME = 'omi-health/sum-small'

In [5]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map='cuda', load_in_8bit=True,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/987 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
dialogue ='''Doctor: Hello, can you please tell me about your past medical history?
Patient: Hi, I don't have any significant past medical history.

Doctor: Okay. What brings you in today?
Patient: I've been experiencing a sharp pain in my chest, around the heart area, for the past three days. It's particularly bad when I take deep breaths or lie down.

Doctor: Thank you for sharing that. Have you had any other symptoms such as shortness of breath, palpitations, dizziness, sweating, nausea, or swelling in your legs?
Patient: Yes, I've had some shortness of breath and occasional palpitations. No dizziness, sweating, or leg swelling, though.

Doctor: I see. Have you had any recent infections, injuries, or stressful events? Do you smoke, drink alcohol, or use any illicit drugs?
Patient: No recent infections or injuries, but I have been very stressed at work lately. I don't smoke or use drugs, but I do have a glass of wine occasionally.

Doctor: Alright. We checked your vital signs, and they were normal. During the physical exam, we noted that your heart sounds were normal, but there was a slight pericardial friction rub. Your lungs were clear, and there was no sign of edema in your legs.

Patient: What does that mean?
Doctor: The pericardial friction rub can indicate inflammation of the pericardium, the lining around your heart. This could be causing your chest pain.

Patient: Oh, I see. What could be causing the inflammation?
Doctor: There are several possible causes, including viral infections, autoimmune diseases, and sometimes even unknown reasons. Your admitting labs showed elevated inflammatory markers, with an ESR of 50 mm/hr and a CRP of 20 mg/L. Your cardiac enzymes were within normal limits, which is reassuring.

Patient: So, what’s the next step?
Doctor: We need to conduct further tests, including an electrocardiogram (ECG) and an echocardiogram, to get more detailed information about your heart function and structure. We may also consult with a cardiologist. In the meantime, we’ll provide you with medications to help reduce the inflammation and manage your pain. We’ll keep you informed about any new findings and discuss the best course of treatment.

Patient: Alright, thank you, Doctor.
Doctor: You’re welcome. If you experience any worsening symptoms, such as increased chest pain, severe shortness of breath, or fainting, please let us know immediately. We’ll continue to monitor you closely.'''


In [11]:
prompt_short = f"""Instruct: Create a medical SOAP summary of this dialogue:
        ### Dialogue:
        {dialogue}
        ### Your SOAP Summary:
        """

prompt = f"""
         role: system, content: You are an expert medical professor assisting in the creation of medically accurate SOAP summaries. Please ensure the response follows the structured format: S:, O:, A:, P: without using markdown or special formatting.
         role: user, content:{prompt_short}
        """

In [12]:
 inputs = tokenizer(
    prompt,
    return_tensors="pt",
    return_attention_mask=False
)

In [14]:
inputs = inputs.to('cuda')
outputs = model.generate(**inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)

text = tokenizer.batch_decode(outputs)[0]
print(text[len(prompt):])

     S (Subjective): The patient, a non-smoker with occasional alcohol use, presents with a chief complaint of sharp chest pain localized around the heart area, worsening with deep breaths and lying down. The pain has been persistent for three days. The patient reports associated symptoms of shortness of breath and occasional palpitations but denies dizziness, sweating, or leg swelling. The patient has been under significant stress at work. No recent infections, injuries, or use of illicit drugs reported.
         O (Objective): Vital signs are within normal limits. Physical examination revealed a slight pericardial friction rub, indicating inflammation of the pericardium. Lungs are clear, and there is no edema in the legs. Cardiac enzymes are normal. Elevated inflammatory markers noted with an ESR of 50 mm/hr and a CRP of 20 mg/L.
         A (Assessment): The primary diagnosis is pericarditis, likely due to an inflammatory process, considering the elev
